I dati utilizzati in questo notebook sono stati presi dalla competizione di Kaggle [Twitter Sentiment Analysis](https://www.kaggle.com/c/twitter-sentiment-analysis2).

# Analisi del sentimento

## Indice

1. [Twitter Sentiment Analysis](#twitter)<br>
    1.1 [Descrizione](#descrizione)<br>
2. [Analisi lessicale](#lessicale)<br>
    2.1 [Sostituire pattern specifici](#sostituire)<br>
    2.2 [Ridurre il tweet in *token*](#token)<br>
    2.3 [Rimuovere le *stop word*](#stop_word)<br>
    2.4 [Ridurre i *token* alla radice (*stemming*)](#stemming)<br>
3. [Analisi esplorativa](#esplorativa)<br>
    3.1 [Preparare i dati per l'analisi esplorativa](#preparare)<br>
    3.2 [Visualizzare i *token* e gli *hashtag* più frequenti dividendo tra tweet positivi e negativi](#token_hashtag)<br>
5. [Metriche di classificazione](#metriche)<br>
5. [Classificare i tweet](#classificare)<br>
    5.1 [Creare una baseline](#baseline)<br>
    5.2 [Creare una pipeline di classificazione](#pipeline)<br>  
6. [Analizzare la performance del modello](#performance)<br>
7. [Analizzare il modello stimato](#analizzare_modello)<br>
8. [Analizzare gli errori di previsione](#errori)<br>

In [ ]:
import inspect
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

# 1. [Twitter Sentiment Analysis](https://www.kaggle.com/c/twitter-sentiment-analysis2) <a id=twitter> </a>

## 1.1 Descrizione <a id=descrizione> </a>

### Description
This contest is taken from the real task of Text Processing.

The task is to build a model that will determine the tone (neutral, positive, negative) of the text. To do this, you will need to train the model on the existing data (train.csv). The resulting model will have to determine the class (neutral, positive, negative) of new texts (test data that were not used to build the model) with maximum accuracy.

> Nota: la descrizione parla di tre classi ma nel dataset sono presenti solo due classi. La metrica nella descrizione sembra essere l'accuratezza ma in Evaluation sembra invece essere l'F1 score. Noi consideriamo il problema come di classificazione binario e utilizzeremo come metrica principale l'F1 score.

### Evaluation
The evaluation metric for this competition is Mean F1-Score. The F1 score, commonly used in information retrieval, measures accuracy using the statistics precision p and recall r. Precision is the ratio of true positives (tp) to all predicted positives (tp + fp). Recall is the ratio of true positives to all actual positives (tp + fn). The F1 score is given by:
$$
F1 = 2\frac{p \cdot r}{p + r}\, \text{where}\, p = \frac{tp}{tp + fp},\,  r = \frac{tp}{tp + fn}
$$
The F1 metric weights recall and precision equally, and a good retrieval algorithm will maximize both precision and recall simultaneously. Thus, moderately good performance on both will be favored over extremely good performance on one and poor performance on the other.

### Leggere i dati

In [ ]:
PATH = "datasets/twitter"

dati = pd.read_csv(PATH + "/train.csv", encoding="latin")
print("Dimensione del dataset: {} x {}".format(*dati.shape))
dati.head()

### Dividere le variabili esplicative dalla variabile risposta

In [ ]:
X, y = dati["SentimentText"].tolist(), dati["Sentiment"].values

#  2. Analisi lessicale <a id=lessicale> </a>

## 2.1 Sostituire pattern specifici <a id=sostituire> </a>

### Sostituire i tag HTML

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
tweet = X[91]
print("Tweet:\n{}".format(tweet))
print("\nTweet dopo aver sostituito i tag HTML:\n{}".format(BeautifulSoup(tweet, "lxml").get_text()))

### Sostituire i collegamenti ipertestuali

In [ ]:
import re

In [ ]:
tweet = X[16]
print("Tweet:\n{}".format(tweet))
print("\nTweet dopo aver sostituito i collegamenti ipertestuali:\n{}".format(re.sub("http\S+", " link ", tweet)))

## 2.2 Ridurre il tweet in *token* <a id=token> </a>

In [ ]:
from nltk.tokenize import TweetTokenizer

In [ ]:
tokenizer = TweetTokenizer(
    preserve_case=False, # se False: Questo è un ESEMPIO -> ['questo', 'è', 'un', 'esempio']
    reduce_len=True, # se True: ma daiiiii non ci credooooo -> ['ma', 'daiii', 'non', 'ci', 'credooo']
    strip_handles=True # se True: cosa ne pensi @mario? -> ['cosa', 'ne', 'pensi', '?']
)

tweet = X[2715]
print("Tweet:\n{}".format(tweet))
print("\nTweet dopo la riduzione in token:\n{}".format(tokenizer.tokenize(tweet)))

## 2.3 Rimuovere le *stop word* <a id=stop_word> </a>

In [ ]:
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
from string import punctuation

### Rimuovere alcune *stop word* predefinite e la punteggiatura

In [ ]:
stop_words = stopwords.words('english') + list(punctuation)

tweet = X[0]
tweet = tokenizer.tokenize(tweet)
print("Tweet dopo la riduzione in token:\n{}".format(tweet))
print("\nTweet dopo la rimozione delle stop words:\n{}".format([token for token in tweet if token not in stop_words]))

### Rimuovere i numeri

In [ ]:
tweet = X[3]
tweet = tokenizer.tokenize(tweet)
print("Tweet dopo la riduzione in token:\n{}".format(tweet))
print("\nTweet dopo la rimozione delle stop words:\n{}".format([token for token in tweet if not token.isdigit()]))

## 2.4 Ridurre i *token* alla radice (*stemming*) <a id=stemming> </a>

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer("english")

tweet = X[1]
tweet = tokenizer.tokenize(tweet)
print("Tweet dopo la riduzione in token:\n{}".format(tweet))
print("\nTweet dopo la riduzione alla radice dei token:\n{}".format([stemmer.stem(token) for token in tweet]))

### Esercizio

1. Completare la funzione `tweet_analyzer()` definita in `msbd/preprocessamento/tweet_analyzer.py`. Sulla traccia di quanto visto finora, la funzione dovrà:
   1. Sostituire i tag HTML e i collegamenti ipertestuali;
   2. Trasformare il tweet in una lista di *token*;
   3. Rimuovere le *stop word* (compresi i numeri come visto sopra);
   5. Ridurre i *token* alla radice.
2. Verificare la corettezza della funzione utilizzando pytest.

In [ ]:
from msbd.preprocessamento import tweet_analyzer

print(inspect.getsource(tweet_analyzer))

In [ ]:
!pytest -v msbd/tests/test_tweet_analyzer.py

### Esempio di tweet dopo il preprocessamento

In [ ]:
tweet = "@student! analyze this &lt;3 tweeeet;, solution at http://www.fakelink.com :D 42 #42"
print("Tweet:\n{}".format(tweet))
print("\nTweet dopo la riduzione alla radice dei token:\n{}".format(tweet_analyzer(tweet, tokenizer, stemmer, stop_words)))

# 3. Analisi esplorativa <a id=esplorativa> </a>

### Dividiere i dati in training e test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print("# tweet in train: {} ({} pos / {} neg)".format(len(X_train), (y_train == 1).sum(), (y_train == 0).sum()))
print("# tweet in test: {}".format(len(X_test)))

## 3.1 Preparare i dati per l'analisi esplorativa <a id=preparare> </a>

### Preprocessare i tweet

In [ ]:
import tqdm

In [ ]:
X_preproc = [tweet_analyzer(tweet, tokenizer, stemmer, stop_words) for tweet in tqdm.tqdm(X_train)]

### Creare le liste dei *token* appartenenti a tweet con sentimento postivo e negativo

In [ ]:
import itertools

In [ ]:
token_pos = list(itertools.chain.from_iterable(list(itertools.compress(X_preproc, y_train == 1))))
token_neg = list(itertools.chain.from_iterable(list(itertools.compress(X_preproc, y_train == 0))))

### Creare le liste degli *hashtag* appartenenti a tweet con sentimento postivo e negativo

In [ ]:
hashtag_pos = [token for token in token_pos if token.startswith("#")]
hashtag_neg = [token for token in token_neg if token.startswith("#")]

## 3.2 Visualizzare i *token* e gli *hashtag* più frequenti dividendo tra tweet positivi e negativi <a id=token_hashtag> </a>

### Creare un'istanza della classe `Counter` per ogni lista

In [ ]:
from collections import Counter

In [ ]:
c_token_pos = Counter(token_pos)
c_token_neg = Counter(token_neg)
c_hashtag_pos = Counter(hashtag_pos)
c_hashtag_neg = Counter(hashtag_neg)

### Grafici a barre

In [ ]:
N = 5

plt.figure(figsize=(15, 3))

plt.subplot(121)
plt.title("{} hashtag più frequenti nei tweet positivi".format(N))
plt.bar(*zip(*c_hashtag_pos.most_common(N)), color="gold")
plt.xticks(rotation="vertical")

plt.subplot(122)
plt.title("{} hashtag più frequenti nei tweet negativi".format(N))
plt.bar(*zip(*c_hashtag_neg.most_common(N)), color="midnightblue")
plt.xticks(rotation="vertical")

plt.show()

In [ ]:
N = 20

plt.figure(figsize=(15, 3))

plt.subplot(121)
plt.title("{} token più frequenti nei tweet positivi".format(N))
plt.bar(*zip(*c_token_pos.most_common(N)), color="gold")
plt.xticks(rotation="vertical")

plt.subplot(122)
plt.title("{} token più frequenti nei tweet negativi".format(N))
plt.bar(*zip(*c_token_neg.most_common(N)), color="midnightblue")
plt.xticks(rotation="vertical")

plt.show()

### Nuvole di parole

In [ ]:
from wordcloud import WordCloud

In [ ]:
MASK = plt.imread("figures/twitter.jpg")
MAX_WORDS = 200
MAX_FONT_SIZE = 200
RELATIVE_SCALING = 1


wc_pos = WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
).generate_from_frequencies(c_token_pos)

wc_neg = WordCloud(
    mask=MASK[:, ::-1, :],
    max_words=MAX_WORDS,
    background_color="midnightblue",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
    colormap=plt.cm.YlOrRd
).generate_from_frequencies(c_token_neg)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.imshow(wc_pos, interpolation='bilinear')
plt.axis("off")

plt.subplot(122)
plt.imshow(wc_neg, interpolation='bilinear')
plt.axis("off")

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

# 4. Metriche di classificazione <a id=metriche> </a>

### Matrice di confusione e metriche derivabili da essa

![confusion_matrix](figures/confusion_matrix.png)

*Immagine presa dalla pagina [Confusion_matrix](https://en.wikipedia.org/wiki/Confusion_matrix) di Wikipedia.*

### Esercizio

1. Completare i metodi della classe `MetricheClassificazione` definita in `msbd/preprocessamento/metriche.py`;
2. Verificare la corettezza dei metodi definiti utilizzando pytest.

> Suggerimenti: 
> 1. Prendere ispirazione dai metodi già definiti;
> 2. Eseguire il controllo con pytest ogni volta che si definisce un nuovo metodo.

In [ ]:
from msbd.metriche import MetricheClassificazione

print(inspect.getsource(MetricheClassificazione))

In [ ]:
!pytest -v msbd/tests/test_metriche_classificazione.py

### Esempio

In [ ]:
y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
y_pred = np.array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1])

print("# negativi: {}".format(MetricheClassificazione.n_negativi(y_true, y_pred)))
print("# positivi: {}".format(MetricheClassificazione.n_positivi(y_true, y_pred)))
print("# previsti negativi: {}".format(MetricheClassificazione.n_previsti_negativi(y_true, y_pred)))
print("# previsti positivi: {}".format(MetricheClassificazione.n_previsti_positivi(y_true, y_pred)))
print()
print("Matrice di confusione:")
print("# veri negativi: {}".format(MetricheClassificazione.n_veri_negativi(y_true, y_pred)))
print("# falsi positivi: {}".format(MetricheClassificazione.n_falsi_positivi(y_true, y_pred)))
print("# falsi negativi: {}".format(MetricheClassificazione.n_falsi_negativi(y_true, y_pred)))
print("# veri positivi: {}".format(MetricheClassificazione.n_veri_positivi(y_true, y_pred)))
print()
print("Tasso falsi positivi: {:.2f}".format(MetricheClassificazione.tasso_falsi_positivi(y_true, y_pred)))
print("Tasso veri positivi: {:.2f}".format(MetricheClassificazione.tasso_veri_positivi(y_true, y_pred)))
print("Precisione: {:.2f}".format(MetricheClassificazione.precisione(y_true, y_pred)))
print("Richiamo: {:.2f}".format(MetricheClassificazione.richiamo(y_true, y_pred)))
print("Punteggio F1: {:.2f}".format(MetricheClassificazione.punteggio_f1(y_true, y_pred)))

# 5. Classificare i tweet <a id=classificare> </a>

## 5.1 Creare una baseline <a id=baseline> </a>

In [ ]:
from msbd.grafici import grafico_matrice_confusione
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
dc = DummyClassifier(strategy="most_frequent")

dc.fit(X_train, y_train)

y_pred = dc.predict(X_test)

precisione_baseline = precision_score(y_test, y_pred)
richiamo_baseline = recall_score(y_test, y_pred)
f1_score_baseline = f1_score(y_test, y_pred)
print("Precisione: {:.2f}".format(precisione_baseline))
print("Richiamo: {:.2f}".format(richiamo_baseline))
print("F1 score: {:.2f}".format(f1_score_baseline))
grafico_matrice_confusione(y_test, y_pred, ["neg", "pos"])

### Esercizio

`DummyClassifier` ha un F1 score del 73% e un richiamo addirittura del 100%! Ѐ utile in un caso reale la previsione fatta da questo modello? Motivare la risposta e riflettere sul risultato.

# 5.2 Creare una pipeline di classificazione <a id=pipeline> </a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

### Definire la pipeline

In [ ]:
vect = CountVectorizer(
    analyzer=lambda t: tweet_analyzer(t, tokenizer, stemmer, stop_words),
    min_df=50,
    max_df=0.7,
)
tree = DecisionTreeClassifier(min_samples_leaf=25)

clf = Pipeline([('vect', vect), ('tree', tree)])

clf.fit(X_train, y_train)

> Nota: tutti gli iperparametri sono stati scelti "a priori" e, sopratutto, senza prendere decisioni basate sull'insieme di *test*. Volendo scegliere la combinazione di iperparametri migliore tra un insieme di candidati (vedi *grid search*, *random search*, ...), avremmo bisogno anche di un terzo insieme di *validation*. Lo stesso vale per la scelta tra algoritmi diversi (es: `DecisionTreeClassifier`vs `LogisticRegression`).

# 6. Analizzare la performance del modello <a id=performance> </a>

### Stimare, per ogni tweet del test set, la probabilità che il suo sentimento sia positivo

In [ ]:
SOGLIA_DECISIONALE = 0.5 # default

y_score = clf.predict_proba(X_test)[:, 1]
y_pred = (y_score > SOGLIA_DECISIONALE).astype(int) # equivalente a y_pred = clf.predict(X_test)

### Esercizio

Descrivere un caso in cui la soglia decisionale di default (0.5) non è adeguata.

### Analizzare la performance del modello fissata la soglia decisionale

In [ ]:
print("Precisione: {:.2f} (baseline = {:.2f})".format(precision_score(y_test, y_pred), precisione_baseline))
print("Richiamo: {:.2f} (baseline = {:.2f})".format(recall_score(y_test, y_pred), richiamo_baseline))
print("F1 score: {:.2f} (baseline = {:.2f})".format(f1_score(y_test, y_pred), f1_score_baseline))
grafico_matrice_confusione(y_test, y_pred, ["neg", "pos"])

### Analizzare le combinazioni di valori ottenibili per le metriche d'interesse al variare della soglia decisionale

In [ ]:
y_pred_25 = (y_score > 0.25).astype(int)
y_pred_50 = (y_score > 0.5).astype(int)
y_pred_75 = (y_score > 0.75).astype(int)

In [ ]:
from msbd.grafici import grafico_curva_precisione_richiamo
from msbd.grafici import grafico_curva_roc

In [ ]:
MARKER = "*"
S = 100

plt.figure(figsize=(10, 5))

plt.subplot(121)
grafico_curva_roc(y_test, y_score)
plt.scatter(MetricheClassificazione.tasso_falsi_positivi(y_test, y_pred_75), recall_score(y_test, y_pred_75), 
            marker=MARKER, s=S, c="brown", label="Soglia decisionale = 0.75", zorder=3)
plt.scatter(MetricheClassificazione.tasso_falsi_positivi(y_test, y_pred_50), recall_score(y_test, y_pred_50), 
            marker=MARKER, s=S, c="red", label="Soglia decisionale = 0.5", zorder=3)
plt.scatter(MetricheClassificazione.tasso_falsi_positivi(y_test, y_pred_25), recall_score(y_test, y_pred_25), 
            marker=MARKER, s=S, c="tomato", label="Soglia decisionale = 0.25", zorder=3)
plt.legend()

plt.subplot(122)
grafico_curva_precisione_richiamo(y_test, y_score)
plt.scatter(recall_score(y_test, y_pred_75), precision_score(y_test, y_pred_75), 
            marker=MARKER, s=S, c="brown", label="Soglia decisionale = 0.75", zorder=3)
plt.scatter(recall_score(y_test, y_pred_50), precision_score(y_test, y_pred_50), 
            marker=MARKER, s=S, c="red", label="Soglia decisionale = 0.5", zorder=3)
plt.scatter(recall_score(y_test, y_pred_25), precision_score(y_test, y_pred_25), 
            marker=MARKER, s=S, c="tomato", label="Soglia decisionale = 0.25", zorder=3)
plt.legend()

plt.show()

# 7. Analizzare il modello stimato <a id=analizzare_modello> </a>

### Visualizzare l'albero

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
dot_data = export_graphviz(
    decision_tree=clf.named_steps["tree"], 
    max_depth=4,
    feature_names=clf.named_steps["vect"].get_feature_names(),
    class_names=("Neg", "Pos"),
    filled=True,
    rounded=True,
)
display(graphviz.Source(dot_data))

### Visualizzare l'importanza delle variabili

In [ ]:
from msbd.grafici import grafico_importanza_variabili

In [ ]:
MAX_NUM = 50

plt.figure(figsize=(15, 3))

variabili = clf.named_steps["vect"].get_feature_names()
importanze = clf.named_steps["tree"].feature_importances_

titolo = "Importanza delle prime {} variabili su {}".format(MAX_NUM, len(variabili))

grafico_importanza_variabili(importanze, variabili, max_num=MAX_NUM, titolo=titolo)

plt.show()

# 8. Analizzare gli errori di previsione <a id=errori> </a>

In [ ]:
X_test_preproc = [tweet_analyzer(tweet, tokenizer, stemmer, stop_words) for tweet in tqdm.tqdm(X_test)]
tweet_score = pd.DataFrame({"tweet":X_test, "tweet_preproc": X_test_preproc, "score": y_score, 
                            "sentimento": y_test})

### Vero sentimento negativo, previsto positivo con elevata confidenza

In [ ]:
N = 5

print("Primi {} tweet con sentimento negativo previsti con sentimento positivo:".format(N))

for _, riga in tweet_score[tweet_score["sentimento"] == 0].sort_values("score", ascending=False).head(N).iterrows():
    print("\nScore: {:.2f}".format(riga["score"]))
    print("Tweet:\n{}".format(riga["tweet"]))
    print("Tweet dopo il preprocessamento:\n{}".format(riga["tweet_preproc"]))

### Vero sentimento positivo, previsto negativo con elevata confidenza

### Esercizio

Analizzare il caso in cui il vero sentimento era positivo ma il modello lo ha previsto negativo con elevata confidenza.

In [ ]:
N = 5

print("Primi {} tweet con sentimento positivo previsti con sentimento negativo:".format(N))

# ============== YOUR CODE HERE ==============
raise NotImplementedError
# ============================================